In [3]:
%pip install qiskit
%pip install qiskit_machine_learning
%pip install qiskit_algorithms

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Data management
import numpy as np
import pandas as pd

# Maths
import math
from math import pi

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

# ML
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

# Additional imports
import pylab as pl
from random import random
from numpy import linalg
from qiskit_machine_learning.datasets import ad_hoc_data
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute

# Plot configuration
%matplotlib inline
sns.set_theme()
sns.set_context("poster")
sns.set_style("ticks")

In [ ]:
class Perceptron_Space:
    """
    Perceptron classifier trained using a set of separators in feature space.

    Parameters:
    -----------
    separators : array-like
        Set of separators.
    nb_ampli : int, optional
        Amplification parameter for quantum search. Defaults to 10.
    quantum : bool, optional
        If True, use quantum search for finding the best separator. Defaults to False.
    opti : bool, optional
        If True, use optimized Grover's algorithm for quantum search. Defaults to False.
    """
    """
    nb_hyperplanes: el número de hiperplanos.
    nb_ampli: un parámetro de amplificación, por defecto es 10.
    quantum: un indicador booleano para especificar si se usa la búsqueda cuántica, por defecto es False.
    opti: un indicador booleano para especificar si se usa la optimización de Grover, por defecto es False.
    """
    def __init__(self, nb_hyperplanes, nb_ampli=10, quantum=False, opti=False):
        self.separators = None  # Set of separators
        self.nb_hyperplanes = nb_hyperplanes
        self.selected = 0  # indicará cuál separador (hiperplano) está actualmente seleccionado
        self.n_iter_ = 0  # Number of steps/almacenará los coeficientes del hiperplano predeterminado
        self.coef_ = None  # Default hyperplane
        self.quantum = quantum  # If we use the quantum search
        self.nb_ampli = nb_ampli  # The amplification parameter for the quantum search
        self.opti = opti  # If we use the opti Grover

#X: los datos de entrada (características).
#y: las etiquetas (clases) correspondientes a los datos de entrada.

    def fit(self, X, y):
        """
        Train the perceptron classifier.

        Parameters:
        -----------
        X : array-like of shape (n_samples, n_features)
            Training data.
        y : array-like of shape (n_samples,)
            Target labels.

        Returns:
        --------
        int
            The number of iterations.

        One of the "separators" will be chosen.
        """
        if self.separators is None:
            len_data = len(X[0]) #obtiene la longitud de los datos de entrada
            """
             inicializa los separadores generando nb_hyperplanes vectores aleatorios con una distribución normal multivariante. 
             Cada vector tiene len_data dimensiones. 
             Los vectores son generados alrededor del origen [0]*len_data con una matriz de covarianza identidad np.eye(len_data).
            """
            self.separators = np.random.multivariate_normal([0]*len_data,np.eye(len_data),size=self.nb_hyperplanes)
            np.random.shuffle(self.separators) #mezcla aleatoriamente los separadores generados.
            self.coef_ = self.separators[self.selected]
            
        X_ = np.array([[j for j in i] for i in X]) #lista de listas para que cada sublista quede como una lista de elementos individuales
        y_ = np.array([1 if i == 1 else -1 for i in y])  # Security to ensure that the classes are {-1,1} and not {0,1}.

        # Oracle over the hyerplanes
        """
        crea una lista llamada oracle, donde cada elemento indica si un separador (hiperplano) clasifica correctamente todos los datos. 
        Para cada separador k, se verifica si y_[i] * X_[i, :].dot(self.separators[k]) > 0 para todos los i. Si todas las condiciones se cumplen, 
        se añade un 1 al oracle, de lo contrario, un 0.
        """
        oracle = [int(all([y_[i] * X_[i, :].dot(self.separators[k]) > 0 for i in range(len(X_))])) for k in range(len(self.separators))]

        # Search , 
        #k, steps = classical_search(oracle) if not self.quantum else quantum_search(oracle, nb_ampli=self.nb_ampli, opti=self.opti)
        #la dejo como quantum search
        k, steps = quantum_search(oracle, nb_ampli=self.nb_ampli, opti=self.opti)

        self.n_iter_ += steps * len(X_)  # The number of steps is multiplied by the complexity of the oracle.
        if all([y_[i] * X_[i, :].dot(self.separators[k]) > 0 for i in range(len(X_))]):  # If the search is successful we take the hyperplane.Si es así, actualiza self.selected y self.coef_ con el separador encontrado.
            self.selected = k
            self.coef_ = self.separators[self.selected]

        return self.n_iter_

    def predict(self, X):
        """
        Predict class labels for samples in X.

        Parameters:
        -----------
        X : array-like of shape (n_samples, n_features)
            Samples.

        Returns:
        --------
        array-like of shape (n_samples,)
            Predicted class labels.
        """
        if self.separators is None:
            len_data = len(X[0]) #obtiene la longitud de los datos de entrada
            #inicializa los separadores (hiperplanos) generando nb_hyperplanes vectores aleatorios con una distribución normal multivariante. 
            # Cada vector tiene len_data dimensiones. 
            # Los vectores son generados alrededor del origen [0]*len_data con una matriz de covarianza identidad np.eye(len_data).
            self.separators = np.random.multivariate_normal([0]*len_data,np.eye(len_data),size=self.nb_hyperplanes)
            np.random.shuffle(self.separators)
            self.coef_ = self.separators[self.selected] #asigna el separador seleccionado, usualmente 0
            """
            Por ultimo usa una lista por comprensión para iterar sobre cada muestra x en X.
            Para cada muestra x, calcula el producto punto x.dot(self.separators[self.selected]) entre x y el separador seleccionado.
            Si el producto punto es mayor que 0, la predicción es 1. De lo contrario, la predicción es -1.
            almacena las predicciones en un array de numpy que se retorna como resultado de la función.
            """
        return np.array([1 if x.dot(self.separators[self.selected]) > 0 else -1 for x in X])

Cambios en perceptron

->La conversión de X y y a numpy arrays puede hacerse de manera más directa y eficiente.
->En lugar de usar una comprensión de lista doble para convertir X, se puede convertir directamente con np.array(X).
->Utilizar operaciones vectorizadas en lugar de bucles for para mejorar el rendimiento. numpy permite operaciones rápidas sobre arrays sin necesidad de bucles explícitos.
->En lugar de utilizar una lista de comprensiones anidadas para calcular el oracle, se puede utilizar operaciones matriciales.
->Simplificar las condiciones de búsqueda y validación.

In [5]:
#Fit tratando de que tenga posible mejora¿?
class PerceptronPropio:

    def __init__(self, nb_hyperplanes, nb_ampli=10, quantum=True, opti=False):
        self.separators = None  # Set of separators
        self.nb_hyperplanes = nb_hyperplanes
        self.selected = 0  # indicará cuál separador (hiperplano) está actualmente seleccionado
        self.n_iter_ = 0  # Number of steps/almacenará los coeficientes del hiperplano predeterminado
        self.coef_ = None  # Default hyperplane
        self.quantum = quantum  # If we use the quantum search
        self.nb_ampli = nb_ampli  # The amplification parameter for the quantum search


    def fit(self, X, y):
        # Convertir X a un numpy array si no lo es
        X_ = np.array(X)
        y_ = np.where(np.array(y) == 1, 1, -1)  # Convertir y a numpy array y asegurar que las clases sean {-1,1}

        if self.separators is None:
            len_data = X.shape[1]  # X debe ser un numpy array, así que shape es más directo
            self.separators = np.random.multivariate_normal(np.zeros(len_data), np.eye(len_data), size=self.nb_hyperplanes)
            np.random.shuffle(self.separators)
            self.coef_ = self.separators[self.selected]

        #X_ = np.array(X)  # Convertir X directamente a numpy array
        #y_ = np.where(np.array(y) == 1, 1, -1)  # Convertir y a numpy array y asegurar que las clases sean {-1,1}

        """
        # se calcula el oraculo con vectores
        # calcula el producto punto de manera vectorizada para todos los datos y separadores.
        # con el axis = 0 se, comprueba si todos los productos son mayores que 0 para cada separador.
        # realiza la multiplicación de matrices entre X_ y la transpuesta de self.separators.
        """
        oracle = np.all(y_[:, np.newaxis] * (X_ @ self.separators.T) > 0, axis=0).astype(int)

        # Search , se quita el clasico y se usa unicamente el cuántico. para probar
        #if not self.quantum:
            #k, steps = classical_search(oracle)
        #else:
        k, steps = quantum_search(oracle, nb_ampli=self.nb_ampli, opti=self.opti)

        self.n_iter_ += steps * len(X_)  # Multiplicar el número de pasos por la complejidad del oráculo y actualiza el iterador

        # Verificar si el hiperplano encontrado clasifica correctamente todos los datos
        if np.all(y_ * (X_ @ self.separators[k]) > 0): #calcula el producto punto entre cada muestra de X_ y el hiperplano self.separators[k]. Esto resulta en un array de puntuaciones.
            """
            Multiplica y_ por las puntuaciones resultantes del producto punto. Si una muestra i está correctamente clasificada por el hiperplano k, el producto y_[i] * (X_[i] @ self.separators[k]) será positivo.
            """
            self.selected = k
            self.coef_ = self.separators[self.selected]

        return self.n_iter_


    def predict(self, X):
        if self.separators is None:
            raise ValueError("Separators are not initialized. Please fit the model first.")

        # probamos vectorizar el proceso
        X_ = np.array(X)  # Convertir X a numpy array si no lo es
        """
        Se utiliza el operador @ de numpy para realizar el producto punto de X con el separador seleccionado (self.coef_), 
        para probar si es más eficiente
        """
        predictions = X_ @ self.coef_  # Realizar el producto punto con el coeficiente seleccionado
        return np.where(predictions > 0, 1, -1)  # Convertir los resultados a 1 o -1 según la condición
